# Pregunta 2
| Autor          |  Carnet  |
|----------------|----------|
|Leonel Guerrero | 18-10638 |

Implementación del perceptrón de Rosenblatt para multiples clases


## Configuración general del algoritmo

En la siguiente celda podrá configurar la cantidad de clases, y los archivos (en formato csv) desde donde se extraerán las instancias de los vectores

In [1]:
# Numero de clases
number_of_classes = 2
# Archivos de datos
files = ['data/LifeSci.csv', 'data/Agri.csv'] # 'data/LifeSci.csv', 'data/Agri.csv'; data/EarthSpace.csv', 'data/MedSci.csv'
# Intervalo de valores para los pesos
interval_of_weights = [-0.05, 0.05]
# Número máximo de épocas
max_number_of_epochs = 100
# Tasas de aprendizaje
learning_rates = [0.001, 0.01, 0.1]

if len(files) != number_of_classes:
    print("Error: the number of files must be equal to the number of classes")
    exit(1)

# Vector de instancias, de la forma [(X,d)]
x: 'list[list[list,list]]' = []

# Construimos el vector de las instancias de todos los archivos, agregando a cada instancia el sesgo
for index, file in enumerate(files):
    with open(file) as f:
        instances = []
        for line in f:
            instance = [1] + [float(x) for x in line.split(',')]
            wanted_output = [1 if index == i else 0 for i in range(number_of_classes)]
            # Guardamos la instancia y el vector de salida deseada (x,d)
            instances.append([instance, wanted_output])

        x += instances

print(len(x))

# Se utilizara la función signo para la transferencia
transfer_function = lambda x: 1 if x >= 0 else 0


3856


### Funciones auxiliares

Por favor ejecute la siguiente celda para cargar las funciones auxiliares que se utilizarán en el algoritmo

In [2]:
def calculate_output(w: 'list[list[float]]', x: 'list[float]') -> 'list[float]':
    """
    Calcula la salida de las k neuronas
    """
    return [transfer_function(sum([w[i][j] * x[j] for j in range(len(x))])) for i in range(len(w))]

def calculate_error(y: 'list[float]', d: 'list[float]') -> 'list[float]':
    """
    Calcula el error de las k neuronas
    """
    return [d[i] - y[i] for i in range(len(y))]

def update_weights(w: 'list[list[float]]', x: 'list[float]', e: 'list[float]', learning_rate: float) -> 'list[list[float]]':
    """
    Actualiza los pesos de las k neuronas
    """
    return [[w[i][j] + learning_rate * e[i] * x[j] for j in range(len(x))] for i in range(len(w))]

def number_of_misclassified_instances(x: 'list[list[list,list]]', w: 'list[list[float]]') -> float:
    """
    Calcula la precisión del perceptrón
    """
    # Calculamos la salida de las k neuronas para cada instancia
    outputs = [calculate_output(w, instance[0]) for instance in x]

    # Calculamos el error de las k neuronas para cada instancia
    errors = [calculate_error(outputs[i], x[i][1]) for i in range(len(x))]

    # Calculamos el número de instancias mal clasificadas
    number_of_misclassified_instances = sum([1 if sum([abs(error) for error in errors[i]]) > 0 else 0 for i in range(len(x))])

    return number_of_misclassified_instances

def calculate_accuracy(number_of_misclassified_instances: float, number_of_instances: float) -> float:
    """
    Calcula la precisión del perceptrón
    """
    return 1 - number_of_misclassified_instances / len(number_of_instances)

def save_results(file: str, epochs: 'list[int]', accuracies: 'list[float]'):
    """
    Guarda los resultados en un archivo, guarda las épocas y la precisión
    """
    with open(file, 'w') as f:
        f.write('Epoch,Accuracy\n')
        for epoch, accuracy in zip(epochs, accuracies):
            f.write(f'{epoch},{accuracy}\n')

## Algoritmo del Perceptrón para multiples clases

In [3]:
import random

def perceptron(learning_rate: float):
    # Inicialización de las listas de épocas y precisión para guardar los resultados
    epochs, accuracies = [], []
    # Algoritmo del perceptrón

    # Inicialización del perceptrón
    # Inicialización de la tasa de aprendizaje, la época inicial y el número máximo de épocas
    n, epoch = learning_rate, 0
    MaxEpoch = max_number_of_epochs
    # Inicialización de la matriz de pesos mas inicialización del sesgo
    w = []
    for _ in range(number_of_classes):
        # list comprehension for initializing the weights with random values
        w.append([random.uniform(interval_of_weights[0], interval_of_weights[1]) for _ in range(len(x[0][0]) + 1)])

    # Calculamos el numero de patrones mal clasificados
    output = calculate_output(w, x[0][0])
    # Calculamos el error
    error = calculate_error(output, x[0][1])
    # Contamos el numero de patrones mal clasificados
    number_of_misclassified_patterns = sum([1 if e != 0 else 0 for e in error])

    # Mientras haya patrones mal clasificados y no se haya alcanzado el número máximo de épocas
    # se actualizan los pesos
    while number_of_misclassified_patterns > 0 and epoch < MaxEpoch:
        # Para cada patron
        for instance in x:
            # Calculamos la salida de las k neuronas
            output = calculate_output(w, instance[0])
            # Calculamos el error
            error = calculate_error(output, instance[1])
            # Actualizamos los pesos
            w = update_weights(w, instance[0], error, n)
        # Calculamos el numero de instancias mal clasificadas
        number_of_misclassified_patterns = number_of_misclassified_instances(x, w)
        print("Número de instancias mal clasificadas: ", number_of_misclassified_patterns)
        print("Epoch: ", epoch)
        # Si no hay patrones mal clasificados, salimos del bucle
        if number_of_misclassified_patterns == 0:
            break

        # Guardamos los resultados
        epochs.append(epoch)
        accuracies.append(calculate_accuracy(number_of_misclassified_patterns, x))

        # Incrementamos el número de épocas
        epoch += 1

    print("Número de épocas: ", epoch)
    accuracy = calculate_accuracy(number_of_misclassified_patterns, x)
    print("Precisión: ", accuracy)
    # Save to file csv
    save_results(f'results/LifeSci-Agri_lr_{learning_rate}.csv', epochs, accuracies)

## Ejecución del perceptrón para multiples tasas de aprendizaje

Se ejecutará el algoritmo para diferentes tasas de aprendizaje, y se mostrará el número de 
épocas que van pasando junto con la cantidad de instancias mal clasificadas

In [ ]:
for learning_rate in learning_rates:
    print(f'Learning rate: {learning_rate}')
    perceptron(learning_rate)
    print('\n')

La siguiente celda es para ejecutar el perceptrón con tasas de aprendizaje especificas

In [ ]:
perceptron(0.01)